In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters
input_size = 28 * 28    # 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transforms.ToTensor())

# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [4]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, input_size)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, input_size)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/5], Step [100/600], Loss: 2.2202
Epoch [1/5], Step [200/600], Loss: 2.1261
Epoch [1/5], Step [300/600], Loss: 2.0164
Epoch [1/5], Step [400/600], Loss: 1.9564
Epoch [1/5], Step [500/600], Loss: 1.8384
Epoch [1/5], Step [600/600], Loss: 1.8302
Epoch [2/5], Step [100/600], Loss: 1.7469
Epoch [2/5], Step [200/600], Loss: 1.7295
Epoch [2/5], Step [300/600], Loss: 1.6477
Epoch [2/5], Step [400/600], Loss: 1.6096
Epoch [2/5], Step [500/600], Loss: 1.4563
Epoch [2/5], Step [600/600], Loss: 1.5085
Epoch [3/5], Step [100/600], Loss: 1.4194
Epoch [3/5], Step [200/600], Loss: 1.3154
Epoch [3/5], Step [300/600], Loss: 1.3733
Epoch [3/5], Step [400/600], Loss: 1.2549
Epoch [3/5], Step [500/600], Loss: 1.3392
Epoch [3/5], Step [600/600], Loss: 1.3361
Epoch [4/5], Step [100/600], Loss: 1.2705
Epoch [4/5], Step [200/600], Loss: 1.2695
Epoch [4/5], Step [300/600], Loss: 1.2684
Epoch [4/5], Step [400/600], Loss: 1.0771
Epoch [4/5], Step [500/600], Loss: 1.1379
Epoch [4/5], Step [600/600], Loss:

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), './data/MNIST_logistic_model.pt')